# Linear Models

Apply a linear classifier, `LogisticRegression()` to the heart disease dataset.

The data is hosted on UCI https://archive.ics.uci.edu/ml/datasets/Heart+Disease

We will need `processed-hungarian.data` from the UCI website (also included on D2L).

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
import warnings
warnings.filterwarnings('ignore') #ignoring some deprication warnings

## Load and prepare the data

### The data 

In [ ]:
data = pd.read_csv('processed.hungarian.data', 
                   na_values='?', 
                   names=[ 'age', 'sex', 'cp', 'trestbps', 'chol', 'fbs',
                            'restecg', 'thalach', 'exang', 'oldpeak', 'slope',
                            'ca', 'thal', 'num'])
data.head()

### Data set interpretation

The first 13 columns are features to predict heart disease, column 14 (num) is the target vector.

num: diagnosis of heart disease (angiographic disease status)  
- Value 0 (no disease): < 50% diameter narrowing
- Value 1 (heart disease) : > 50% diameter narrowing

In [ ]:
data.info()

### Drop columns with many missing values 

In [ ]:
data = data.drop(columns=['slope', 'ca', 'thal'])
data.info()

### Fill missing values with mean

In [ ]:
data = data.fillna(data.mean())
data.info()

In [ ]:
data.describe()

In [ ]:
data['num'].value_counts()

### A pairplot to get an overview

In [ ]:
sns.pairplot(data, hue='num')

In [ ]:
data.groupby(by='num').sex.value_counts()

### Combine the data loading code

In [ ]:
def load_heart_disease():
    '''Load and pre-process heart disease data
    
    assumes processed.hungarian.data file is present.
    
    download from
    https://archive.ics.uci.edu/ml/machine-learning-databases/heart-disease/processed.hungarian.data
    
    return: data(DataFrame)
    
    '''
    
    data = pd.read_csv('processed.hungarian.data', 
                   na_values='?', 
                   names=[ 'age', 'sex', 'cp', 'trestbps', 'chol', 'fbs',
                            'restecg', 'thalach', 'exang', 'oldpeak', 'slope',
                            'ca', 'thal', 'num'])
    
    # drop columns with many missing data
    data = data.drop(columns=['slope', 'ca', 'thal'])
    
    # fill in remaining missing data with mean() per column
    data = data.fillna(data.mean())
    
    return data

In [ ]:
data2 = load_heart_disease()

data.equals(data2)

### Create feature matrix and target vector 

In [ ]:
X = data.drop(columns='num')
y = data['num']
print(X.shape)
print(y.shape)

### Create training and test sets

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    test_size=0.1,
                                                    stratify=y,
                                                    random_state=31)

print(X_train.shape)
print(X_test.shape)

**Note**: what does stratify=y do? This parameter makes sure that the data is split with the same proportion of classes as the original dataset. For example, if 20% of the data is class 0 and 80% is class 1, then the training set and testing set will also have 20% class 0 and 80% class 1.

In [ ]:
y_train.value_counts()

In [ ]:
y_test.value_counts()

We can see that the division of classes is the same for both the training and testing data (63-64% for class 1).

We now set aside the testing data and use the training data to validate the model.

## Apply `LogisticRegression()` and evaluate the performance 

### Cross validation 

Are we over- or underfitting?

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score

model = LogisticRegression(max_iter=1000)
scores = cross_val_score(model, X_train, y_train, cv=5)
scores

In [ ]:
scores.mean()

Using `cross_val_score` only gives us the validation results. To check if we are over- or underfitting, we also need the training results. For this, we can use `cross_validate` instead

In [ ]:
from sklearn.model_selection import cross_validate
scores = cross_validate(model, X_train, y_train, cv=5, 
                        scoring='accuracy',
                       return_train_score=True)
scores

In [ ]:
for label_pair in [ ('train_score', 'train_score'), ('test_score', 'validation_score')]:
    print('{}= {:.3f}'.format(label_pair[1], scores[label_pair[0]].mean()))

Looks pretty close. We are likely underfitting and would need a more complex model to increase validation set accuracy.

Lets try and play with the hyperparameter, `C`. Note that the default value for `C` is 1. To increase the model complexity, we need to increase the value of `C`.

In [ ]:
model = LogisticRegression(C=100, max_iter=1000)
model.fit(X_train, y_train)
scores = cross_validate(model, X_train, y_train, cv=5, 
                        scoring='accuracy',
                       return_train_score=True)
for label_pair in [ ('train_score', 'train_score'), ('test_score', 'validation_score')]:
    print('{}= {:.3f}'.format(label_pair[1], scores[label_pair[0]].mean()))

Does not seem to help. We need a more complex model. We will get back to it later on.

### The confusion matrix on the test set 

Your thoughts?

In [ ]:
from sklearn.metrics import confusion_matrix

y_pred = model.predict(X_test)
mat = confusion_matrix(y_test, y_pred)

sns.heatmap(mat, xticklabels=['no heart disease', 'heart disease'],  yticklabels=['no heart disease', 'heart disease'], square=True, annot=True, cbar=False)
plt.xlabel('predicted value')
plt.ylabel('true value')

There are two cases were the classifier says *no heart disease* but these patients do have a *heart disease*. These are  false negatives - we miss patients with the disease.

There are three cases were the classifier says *heart disease* but these patients do *not* have a *heart disease*. These are false positives - we diagnose disease when there is none.

### Summarize the different scores 

In [ ]:
model = LogisticRegression(max_iter=1000)
model.fit(X_train, y_train)

scores = cross_validate(model, X_train, y_train, cv=5, 
                        scoring='accuracy',
                       return_train_score=True)

print('training accuracy (all data) {:.3f}'.format(model.score(X_train, y_train)))

print('training accuracy (cross-validation) {:.3f}'.format(scores['train_score'].mean()))

print('validation accuracy (cross-validation) {:.3f}'.format(scores['test_score'].mean()))

print('test accuracy (new data) {:.3f}'.format(model.score(X_test, y_test)))